In [28]:
import math
import numpy as np
from numpy import linalg as la
import pandas as pd

# Boostrap Zero Curve

The idea is that, using a sequence of coupon bonds to solve for zero yield.

Assuming principle is $100, coupons are paid semi-annually.

Suppose we have $n$ bonds, each matures half a year later then the previous one.

     bond1 pays coupon+principle at period 1.

     bond2 pays coupon at period 1, coupon+principle at period 2.
     
     bond3 pays coupon at period 1, coupon at period 2, coupon+principle at period 3.
     etc..

In [19]:
# a total of n bonds and n periods
bond1 = ["03/31/19", 1.5, 100.44, 2.48]
bond2 = ["09/30/19", 1.0, 99.44, 2.52]
bond3 = ["03/31/20", 2.25, 100.67, 2.41]
bond4 = ["09/30/20", 2, 99.93, 2.52]


bond_data = pd.DataFrame([bond1, bond2, bond3, bond4])
bond_data.columns=["maturity", "coupon_rate", "dirty_price", "yield"]
bond_data["maturity"] = pd.to_datetime(bond_data["maturity"])
print(bond_data)
bond_data["coupon_rate"][0]

    maturity  coupon_rate  dirty_price  yield
0 2019-03-31         1.50       100.44   2.48
1 2019-09-30         1.00        99.44   2.52
2 2020-03-31         2.25       100.67   2.41
3 2020-09-30         2.00        99.93   2.52


1.5

In [33]:
# Construct a matrix to represent the incoming payments

def get_matrix(bond_df):
    # this is a n*n matrix
    n = len(bond_df)
    mat = np.zeros((n,n))
    # the ith column represents the incoming payments from the ith bond
    for i in range(n):
        for j in range(i+1):
            # the payment from the ith bond at jth period
            mat[j][i] += bond_df["coupon_rate"][i]/2 # since semi-annully, % * 100
            if j==i:
                # assuming the ith bond matures at the ith period
                mat[j][i] += 100 # assuming priciple = 100
    return mat   



def get_zero_rate(bond_df):
    mat = get_matrix(bond_df)
    vec_p = np.array(bond_df["dirty_price"])
    # we want to find a vector [q1, q2, ..., qn] such that qi is the present value of $1 at period i.
    # we have 
    # q@A = p, where both q and p are row vectors, and p is the dirty price
    
    # q = p@A.inverse
    
    return vec_p @ la.inv(mat)
    
    
    


get_zero_rate(bond_data)


array([0.99692308, 0.98449292, 0.97345767, 0.96014977])